In [1]:
import nest_asyncio
nest_asyncio.apply()
from ib_insync import *
import pandas as pd
import os
from datetime import datetime

# Define async wrapper
async def run_live_stream():
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=11)

    contract = Stock('AAPL', 'SMART', 'USD')
    ib.reqMktData(contract, '', False, False)

    csv_file = 'AAPL_5_seconds_data.csv'

    def on_tick(tick):
        row = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'),
            'bid': contract.bid,
            'ask': contract.ask,
            'last': contract.last,
            'volume': contract.volume
        }
        df = pd.DataFrame([row])
        if not os.path.exists(csv_file):
            df.to_csv(csv_file, mode='w', index=False)
        else:
            df.to_csv(csv_file, mode='a', index=False, header=False)

    ib.pendingTickersEvent += on_tick

    # Run event loop for 10 seconds
    await ib.sleep(10)

    ib.cancelMktData(contract)
    ib.disconnect()

# In IPython or Jupyter, run the async function:
await run_live_stream()


Error 10197, reqId 3: No market data during competing live session, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


TypeError: object bool can't be used in 'await' expression

In [2]:
ib.disconnect()